In [1]:
import os
import re
import glob
from datetime import datetime, time
from collections import Counter

import pandas as pd

os.makedirs("youtube/data/csv", exist_ok=True)

def extract_country_from_filename(path):
    name = os.path.basename(path)
    name_no_ext = os.path.splitext(name)[0]  # remove extension
    
    # Try matching first 2 uppercase letters at the start
    m = re.match(r'^([A-Za-z]{2})', name_no_ext)
    if m:
        return m.group(1).upper()
    
    # Otherwise, look for any 2 letter part in the name
    parts = re.split(r'[^A-Za-z]+', name_no_ext)
    for p in parts:
        if p and len(p) < 3:
            return p.upper()
    
    # fallback to generic "UNKNOWN"
    #return "UNKNOWN"


def parse_publish_time(s):
    if pd.isna(s):
        return None
    if isinstance(s, (datetime,)):
        return s.time()
    s = str(s)
    try:
        dt = datetime.fromisoformat(s.replace('Z', '+00:00'))
        return dt.time()
    except Exception:
        pass
    for fmt in ("%H:%M:%S", "%H:%M"):
        try:
            return datetime.strptime(s, fmt).time()
        except Exception:
            pass
    m = re.search(r'(\d{1,2}:\d{2})', s)
    if m:
        try:
            return datetime.strptime(m.group(1), "%H:%M").time()
        except Exception:
            pass
    return None


def bucket_time_10min(t: time):
    if t is None:
        return None
    minutes = t.hour * 60 + t.minute
    bucket_start = (minutes // 10) * 10
    start_h = bucket_start // 60
    start_m = bucket_start % 60
    end_minutes = bucket_start + 10
    end_h = end_minutes // 60
    end_m = end_minutes % 60
    return f"{start_h:02d}:{start_m:02d}-{end_h:02d}:{end_m:02d}"

In [ ]:
#1.Create a single dataframe with the concatenation of all input csv files, adding a column called country

In [2]:
csv_dir = r'C:\Users\Utente\youtube\data\csv'
all_csv_paths = sorted(glob.glob(os.path.join(csv_dir, '*.csv')))
#print(f"Number of CSV files: {len(all_csv_paths)}")
all_csv_paths

['C:\\Users\\Utente\\youtube\\data\\csv\\CAvideos.csv',
 'C:\\Users\\Utente\\youtube\\data\\csv\\DEvideos.csv',
 'C:\\Users\\Utente\\youtube\\data\\csv\\FRvideos.csv',
 'C:\\Users\\Utente\\youtube\\data\\csv\\GBvideos.csv',
 'C:\\Users\\Utente\\youtube\\data\\csv\\INvideos.csv',
 'C:\\Users\\Utente\\youtube\\data\\csv\\JPvideos.csv',
 'C:\\Users\\Utente\\youtube\\data\\csv\\KRvideos.csv',
 'C:\\Users\\Utente\\youtube\\data\\csv\\MXvideos.csv',
 'C:\\Users\\Utente\\youtube\\data\\csv\\RUvideos.csv',
 'C:\\Users\\Utente\\youtube\\data\\csv\\USvideos.csv']

In [3]:
dfs = []

for path in all_csv_paths:
    try:
        # Try reading with 'utf-8', if fails, fall back to 'latin1'
        try:
            df = pd.read_csv(path, encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(path, encoding='latin1')
        
        df['country'] = extract_country_from_filename(path)
        dfs.append(df)
    except Exception as e:
        print(f"Error reading {path}: {e}")

# Concatenate all successfully read CSVs
final_df = pd.concat(dfs, ignore_index=True)
print(final_df.head())

      video_id trending_date  \
0  n1WpP7iowLc      17.14.11   
1  0dBIkQ4Mz1M      17.14.11   
2  5qpjK5DgCt4      17.14.11   
3  d380meD0W0M      17.14.11   
4  2Vv-BfVoq4g      17.14.11   

                                               title channel_title  \
0         Eminem - Walk On Water (Audio) ft. Beyoncé    EminemVEVO   
1                      PLUSH - Bad Unboxing Fan Mail     iDubbbzTV   
2  Racist Superman | Rudy Mancuso, King Bach & Le...  Rudy Mancuso   
3                           I Dare You: GOING BALD!?      nigahiga   
4        Ed Sheeran - Perfect (Official Music Video)    Ed Sheeran   

   category_id              publish_time  \
0           10  2017-11-10T17:00:03.000Z   
1           23  2017-11-13T17:00:00.000Z   
2           23  2017-11-12T19:05:24.000Z   
3           24  2017-11-12T18:01:41.000Z   
4           10  2017-11-09T11:04:14.000Z   

                                                tags     views    likes  \
0  Eminem|"Walk"|"On"|"Water"|"Aftermath/Shady

In [ ]:
#6.Cluster the publish time into 10-minute intervals (e.g. from 02:20 to 02:30)

In [4]:
final_df['publish_time_parsed'] = final_df['publish_time'].apply(parse_publish_time)
final_df['publish_time_bucket'] = final_df['publish_time_parsed'].apply(bucket_time_10min)
print(final_df[['publish_time', 'publish_time_parsed', 'publish_time_bucket']].head())

               publish_time publish_time_parsed publish_time_bucket
0  2017-11-10T17:00:03.000Z            17:00:03         17:00-17:10
1  2017-11-13T17:00:00.000Z            17:00:00         17:00-17:10
2  2017-11-12T19:05:24.000Z            19:05:24         19:00-19:10
3  2017-11-12T18:01:41.000Z            18:01:41         18:00-18:10
4  2017-11-09T11:04:14.000Z            11:04:14         11:00-11:10


In [ ]:
#7.For each interval, determine the number of videos, average number of likes and of dislikes.

In [5]:
interval_stats = final_df.groupby('publish_time_bucket').agg(
    num_videos=('video_id', 'count'),
    avg_likes=('likes', 'mean'),
    avg_dislikes=('dislikes', 'mean')
).reset_index()
print(interval_stats.head())

  publish_time_bucket  num_videos     avg_likes  avg_dislikes
0         00:00-00:10        2913  60951.483350   3787.232750
1         00:10-00:20        1522  22553.870565   1437.457293
2         00:20-00:30        1248  21258.370192   1066.330128
3         00:30-00:40        1625  36604.352000    949.439385
4         00:40-00:50        1283  41770.614186   1889.012471


In [ ]:
#8.For each tag, determine the number of videos. Notice that tags contains a string with several tags.

In [7]:
all_tags = []

for tags_string in final_df['tags']:
    if pd.notna(tags_string):  # Check if not NaN
        # Split by pipe and clean each tag
        tags_list = str(tags_string).split('|')
        # Remove quotes and whitespace from each tag
        cleaned_tags = [tag.strip().strip('"') for tag in tags_list if tag.strip()]
        all_tags.extend(cleaned_tags) #extend: add tags one by one 

# Count occurrences of each tag
tag_counts = Counter(all_tags)
#converting to a dataframe to print it
tag_stats = pd.DataFrame(tag_counts.items(), columns=['tag', 'num_tag_used'])
print(f"Total unique tags: {len(tag_counts)}")
print("\nTop 10 most used tags with the numbers of repeat:")
print(tag_stats.head(10))

Total unique tags: 865696

Top 10 most used tags with the numbers of repeat:
                          tag  num_tag_used
0                      Eminem           462
1                        Walk           139
2                          On            96
3                       Water           109
4  Aftermath/Shady/Interscope           127
5                         Rap          2299
6                       plush            19
7                bad unboxing            20
8                    unboxing          1260
9                    fan mail            39


In [ ]:
#9. Find the tags with the largest number of videos

In [8]:
tag_stats = tag_stats.sort_values('num_tag_used', ascending=False)
print(tag_stats.head(20))

               tag  num_tag_used
822         [none]         37698
51           funny         15075
50          comedy         12356
8292          2018         11402
726           news          6415
153          music          5918
1189          2017          5698
472          video          5640
327          humor          5058
310     television          4174
680           show          4158
373         review          4068
112           vlog          3979
2125           Pop          3963
3235     interview          3861
57            live          3809
531           food          3688
1302  funny videos          3610
862       comedian          3601
456             tv          3453


In [8]:
#10. For each (tag, country) pair, compute average ratio likes/dislikes

In [ ]:
df = final_df.copy()

# 1. Clean and split tags into lists 
df['tag_list'] = df['tags'].apply(
    lambda x: [t.strip().strip('"') for t in str(x).split('|') if t.strip()] 
    if pd.notna(x) else []
)

# 2. Explode so each tag has its own row
df_exploded = df.explode('tag_list')

# 3. Compute ratio likes/dislikes
# Avoid division by zero → replace dislikes = 0 with 1 to calculate number of likes for those videos
df_exploded['dislikes'] = df_exploded['dislikes'].replace(0, 1)

df_exploded['like/dislike_ratio'] = df_exploded['likes'] / df_exploded['dislikes']

# 4. Group by tag and country and compute the average ratio
avg_ratio = (
    df_exploded.groupby(['tag_list', 'country'])['like/dislike_ratio']
    .mean()
    .reset_index()
    .rename(columns={'tag_list': 'tag', 'like/dislike_ratio': 'avg_like/dislike_ratio'})
)

avg_ratio.head(20)